In [1]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [2]:
genome_mm10, genome_129_CAST = {}, {}

for record in SeqIO.parse('/DATA/projects/bvs_alleles/genomes/Mus_musculus.GRCm38.dna_sm.toplevel.fa', 'fasta'):
    genome_mm10[record.id] = str(record.seq)
    
for record in SeqIO.parse('/DATA/projects/bvs_alleles/GRCm38_129_CAST_snpsonly.fa', 'fasta'):
    genome_129_CAST[record.id] = str(record.seq)

In [3]:
def extract_sequence_for_region(genome, region):
    if region[1] == '':
        return(str(genome[region[0].replace('chr', '')][:int(region[2])]))
    elif region[2] == '':
        return(str(genome[region[0].replace('chr', '')][int(region[1]):]))
    else:
        return(str(genome[region[0].replace('chr', '')][int(region[1]):int(region[2])]))

def split_regions(region):
    return([region.split(':')[0], region.split(':')[1].split('-')[0], region.split(':')[1].split('-')[1]])

def get_chrom_sizes(genome_for_output):
    chroms, sizes = [], []
    for record in genome_for_output:
        chroms.append(record.id)
        sizes.append(len(str(record.seq)))
    return(pd.DataFrame({'chroms': chroms, 'sizes': sizes}))

# Find the genomic coordinates of edits

### Deternime coordinates of eGFP/mCherry insertions

In [4]:
sox2_fp_left_flank = 'chr3_129:34651172-34651372'
sox2_fp_right_flank = 'chr3_129:34651375-34651575'
sox2_fp_excision = 'chr3_129:34651372-34651375'

sox2_egfp = str(list(SeqIO.parse("/DATA/projects/bvs_alleles/sox2_rcmc_custom/cell_lines_overview/fasta/01_Sox2_GFP_CAST_allele.fa", "fasta"))[0].seq)
sox2_mcherry = str(list(SeqIO.parse("/DATA/projects/bvs_alleles/sox2_rcmc_custom/cell_lines_overview/fasta/01_Sox2_mCherry_129S1_allele.fa", "fasta"))[0].seq)

### Determine coordinates for -116kb LP insertion
* Sox2 Promoter only
* No insert

In [5]:
landing_pad_116kb_left_flank = 'chr3_129:34643482-34644035'
landing_pad_116kb_right_flank = 'chr3_129:34644035-34644328'

landing_pad_116kb_insert_coord = 34644035
landing_pad_116kb_sox2p = str(list(SeqIO.parse("/DATA/projects/bvs_alleles/sox2_rcmc_custom/cell_lines_overview/fasta/02_LP-116kb_Sox2P.fa", "fasta"))[0].seq)
landing_pad_116kb_nans = 'N'*len(landing_pad_116kb_sox2p)

### Determine coordinates for -161kb LP insertion
* Sox2 Promoter + CDS
* Sox2 Promoter only
* No insert

In [6]:
landing_pad_161kb_left_flank = 'chr3_129:34598230-34598480'
landing_pad_161kb_right_flank = 'chr3_129:34598480-34598729'

landing_pad_161kb_insert_coord = 34598480
landing_pad_161kb_sox2p_cds = str(list(SeqIO.parse("/DATA/projects/bvs_alleles/sox2_rcmc_custom/cell_lines_overview/fasta/03_LP-161kb_Sox2P_CDS.fa", "fasta"))[0].seq)
landing_pad_161kb_sox2p = str(list(SeqIO.parse("/DATA/projects/bvs_alleles/sox2_rcmc_custom/cell_lines_overview/fasta/03_LP-161kb_Sox2P.fa", "fasta"))[0].seq)
landing_pad_161kb_nans = 'N'*len(landing_pad_161kb_sox2p)

### Determine coordinates for Sox2 deletion

In [7]:
deletion_left_flank = 'chr3_129:34646645-34647660'
deletion_right_flank = 'chr3_129:34652613-34653611'
deletion_coordinates = 'chr3_129:34647660-34652613'
sox2_deletion = 'N'*(len(sox2_mcherry) + 4950)

# Construct custom genomes

1. F121/9 GFP+/mCh+ = CM1846
2. LP-116kb_Sox2P = CM2167
3. LP-116kb_Sox2P_mChDel = CM2267
4. LP-161kb_Sox2P = CM2094  
5. LP-161kb_Sox2P_mChDel = CM2266
6. LP-161kb_Sox2P_CDS_mChDel = CM2291

### Cell line 1: F121/9 GFP+/mCh+ (CM1846)

In [249]:
genome_129_CAST = {}

for record in SeqIO.parse('/DATA/projects/bvs_alleles/GRCm38_129_CAST_snpsonly.fa', 'fasta'):
    genome_129_CAST[record.id] = str(record.seq)

In [250]:
seq_before_lp161_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:-34598480'))
seq_lp161_129 = landing_pad_161kb_nans
seq_between_lp161_lp116_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34598480-34644035'))
seq_lp116_129 = landing_pad_116kb_nans
seq_between_lp116_mcherry_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34644035-34651372'))
seq_mcherry_129 = sox2_mcherry
seq_after_mcherry_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34651375-'))

custom_chr3_129 = [seq_before_lp161_129, seq_lp161_129, seq_between_lp161_lp116_129, seq_lp116_129, 
                   seq_between_lp116_mcherry_129, seq_mcherry_129, seq_after_mcherry_129]
custom_chr3_129 = ''.join(custom_chr3_129)

In [251]:
genome_129_CAST['3_129'] = custom_chr3_129

In [252]:
seq_before_lp161_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:-34598480'))
seq_lp161_CAST = landing_pad_161kb_nans
seq_between_lp161_lp116_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34598480-34644035'))
seq_lp116_CAST = landing_pad_116kb_nans
seq_between_lp116_egfp_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34644035-34651372'))
seq_egfp_CAST = sox2_egfp
seq_after_egfp_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34651375-'))

custom_chr3_CAST = [seq_before_lp161_CAST, seq_lp161_CAST, seq_between_lp161_lp116_CAST, seq_lp116_CAST, 
                   seq_between_lp116_egfp_CAST, seq_egfp_CAST, seq_after_egfp_CAST]
custom_chr3_CAST = ''.join(custom_chr3_CAST)

In [253]:
genome_129_CAST['3_CAST'] = custom_chr3_CAST

In [255]:
genome_129_CAST_for_output = [SeqRecord(Seq(sequence), id=key, description="") for key, sequence in genome_129_CAST.items()]

with open('/DATA/projects/bvs_alleles/sox2_rcmc/custom_genomes/CM1846_129_CAST_custom.fa', 'w') as handle:
    SeqIO.write(genome_129_CAST_for_output, handle, 'fasta')

### Cell line 2: LP-116kb_Sox2P (CM2167)

In [256]:
genome_129_CAST = {}

for record in SeqIO.parse('/DATA/projects/bvs_alleles/GRCm38_129_CAST_snpsonly.fa', 'fasta'):
    genome_129_CAST[record.id] = str(record.seq)

In [257]:
seq_before_lp161_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:-34598480'))
seq_lp161_129 = landing_pad_161kb_nans
seq_between_lp161_lp116_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34598480-34644035'))
seq_lp116_129 = landing_pad_116kb_sox2p
seq_between_lp116_mcherry_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34644035-34651372'))
seq_mcherry_129 = sox2_mcherry
seq_after_mcherry_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34651375-'))

custom_chr3_129 = [seq_before_lp161_129, seq_lp161_129, seq_between_lp161_lp116_129, seq_lp116_129, 
                   seq_between_lp116_mcherry_129, seq_mcherry_129, seq_after_mcherry_129]
custom_chr3_129 = ''.join(custom_chr3_129)

In [258]:
genome_129_CAST['3_129'] = custom_chr3_129

In [259]:
seq_before_lp161_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:-34598480'))
seq_lp161_CAST = landing_pad_161kb_nans
seq_between_lp161_lp116_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34598480-34644035'))
seq_lp116_CAST = landing_pad_116kb_nans
seq_between_lp116_egfp_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34644035-34651372'))
seq_egfp_CAST = sox2_egfp
seq_after_egfp_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34651375-'))

custom_chr3_CAST = [seq_before_lp161_CAST, seq_lp161_CAST, seq_between_lp161_lp116_CAST, seq_lp116_CAST, 
                   seq_between_lp116_egfp_CAST, seq_egfp_CAST, seq_after_egfp_CAST]
custom_chr3_CAST = ''.join(custom_chr3_CAST)

In [260]:
genome_129_CAST['3_CAST'] = custom_chr3_CAST

In [262]:
genome_129_CAST_for_output = [SeqRecord(Seq(sequence), id=key, description="") for key, sequence in genome_129_CAST.items()]

with open('/DATA/projects/bvs_alleles/sox2_rcmc/custom_genomes/CM2167_129_CAST_custom.fa', 'w') as handle:
    SeqIO.write(genome_129_CAST_for_output, handle, 'fasta')

### Cell line 3: LP-116kb_Sox2P_mChDel (CM2267)

In [279]:
genome_129_CAST = {}

for record in SeqIO.parse('/DATA/projects/bvs_alleles/GRCm38_129_CAST_snpsonly.fa', 'fasta'):
    genome_129_CAST[record.id] = str(record.seq)

In [280]:
seq_before_lp161_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:-34598480'))
seq_lp161_129 = landing_pad_161kb_nans
seq_between_lp161_lp116_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34598480-34644035'))
seq_lp116_129 = landing_pad_116kb_sox2p
seq_between_lp116_sox2_deletion_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34644035-34647660'))
seq_sox2_deletion_129 = sox2_deletion
seq_after_sox2_deletion_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34652613-'))

custom_chr3_129 = [seq_before_lp161_129, seq_lp161_129, seq_between_lp161_lp116_129, seq_lp116_129, 
                   seq_between_lp116_sox2_deletion_129, seq_sox2_deletion_129, seq_after_sox2_deletion_129]
custom_chr3_129 = ''.join(custom_chr3_129)

In [281]:
genome_129_CAST['3_129'] = custom_chr3_129

In [282]:
seq_before_lp161_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:-34598480'))
seq_lp161_CAST = landing_pad_161kb_nans
seq_between_lp161_lp116_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34598480-34644035'))
seq_lp116_CAST = landing_pad_116kb_nans
seq_between_lp116_egfp_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34644035-34651372'))
seq_egfp_CAST = sox2_egfp
seq_after_egfp_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34651375-'))

custom_chr3_CAST = [seq_before_lp161_CAST, seq_lp161_CAST, seq_between_lp161_lp116_CAST, seq_lp116_CAST, 
                   seq_between_lp116_egfp_CAST, seq_egfp_CAST, seq_after_egfp_CAST]
custom_chr3_CAST = ''.join(custom_chr3_CAST)

In [283]:
genome_129_CAST['3_CAST'] = custom_chr3_CAST

In [285]:
genome_129_CAST_for_output = [SeqRecord(Seq(sequence), id=key, description="") for key, sequence in genome_129_CAST.items()]

with open('/DATA/projects/bvs_alleles/sox2_rcmc/custom_genomes/CM2267_129_CAST_custom.fa', 'w') as handle:
    SeqIO.write(genome_129_CAST_for_output, handle, 'fasta')

### Cell line 4: LP-161kb_Sox2P (CM2094)  

In [286]:
genome_129_CAST = {}

for record in SeqIO.parse('/DATA/projects/bvs_alleles/GRCm38_129_CAST_snpsonly.fa', 'fasta'):
    genome_129_CAST[record.id] = str(record.seq)

In [287]:
seq_before_lp161_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:-34598480'))
seq_lp161_129 = landing_pad_161kb_sox2p
seq_between_lp161_lp116_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34598480-34644035'))
seq_lp116_129 = landing_pad_116kb_nans
seq_between_lp116_mcherry_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34644035-34651372'))
seq_mcherry_129 = sox2_mcherry
seq_after_mcherry_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34651375-'))

custom_chr3_129 = [seq_before_lp161_129, seq_lp161_129, seq_between_lp161_lp116_129, seq_lp116_129, 
                   seq_between_lp116_mcherry_129, seq_mcherry_129, seq_after_mcherry_129]
custom_chr3_129 = ''.join(custom_chr3_129)

In [288]:
genome_129_CAST['3_129'] = custom_chr3_129

In [289]:
seq_before_lp161_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:-34598480'))
seq_lp161_CAST = landing_pad_161kb_nans
seq_between_lp161_lp116_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34598480-34644035'))
seq_lp116_CAST = landing_pad_116kb_nans
seq_between_lp116_egfp_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34644035-34651372'))
seq_egfp_CAST = sox2_egfp
seq_after_egfp_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34651375-'))

custom_chr3_CAST = [seq_before_lp161_CAST, seq_lp161_CAST, seq_between_lp161_lp116_CAST, seq_lp116_CAST, 
                   seq_between_lp116_egfp_CAST, seq_egfp_CAST, seq_after_egfp_CAST]
custom_chr3_CAST = ''.join(custom_chr3_CAST)

In [290]:
genome_129_CAST['3_CAST'] = custom_chr3_CAST

In [292]:
genome_129_CAST_for_output = [SeqRecord(Seq(sequence), id=key, description="") for key, sequence in genome_129_CAST.items()]

with open('/DATA/projects/bvs_alleles/sox2_rcmc/custom_genomes/CM2094_129_CAST_custom.fa', 'w') as handle:
    SeqIO.write(genome_129_CAST_for_output, handle, 'fasta')

### Cell line 5: LP-161kb_Sox2P_mChDel (CM2266)

In [304]:
genome_129_CAST = {}

for record in SeqIO.parse('/DATA/projects/bvs_alleles/GRCm38_129_CAST_snpsonly.fa', 'fasta'):
    genome_129_CAST[record.id] = str(record.seq)

In [305]:
seq_before_lp161_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:-34598480'))
seq_lp161_129 = landing_pad_161kb_sox2p
seq_between_lp161_lp116_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34598480-34644035'))
seq_lp116_129 = landing_pad_116kb_nans
seq_between_lp116_sox2_deletion_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34644035-34647660'))
seq_sox2_deletion_129 = sox2_deletion
seq_after_sox2_deletion_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34652613-'))

custom_chr3_129 = [seq_before_lp161_129, seq_lp161_129, seq_between_lp161_lp116_129, seq_lp116_129, 
                   seq_between_lp116_sox2_deletion_129, seq_sox2_deletion_129, seq_after_sox2_deletion_129]
custom_chr3_129 = ''.join(custom_chr3_129)

In [306]:
genome_129_CAST['3_129'] = custom_chr3_129

In [307]:
seq_before_lp161_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:-34598480'))
seq_lp161_CAST = landing_pad_161kb_nans
seq_between_lp161_lp116_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34598480-34644035'))
seq_lp116_CAST = landing_pad_116kb_nans
seq_between_lp116_egfp_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34644035-34651372'))
seq_egfp_CAST = sox2_egfp
seq_after_egfp_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34651375-'))

custom_chr3_CAST = [seq_before_lp161_CAST, seq_lp161_CAST, seq_between_lp161_lp116_CAST, seq_lp116_CAST, 
                   seq_between_lp116_egfp_CAST, seq_egfp_CAST, seq_after_egfp_CAST]
custom_chr3_CAST = ''.join(custom_chr3_CAST)

In [308]:
genome_129_CAST['3_CAST'] = custom_chr3_CAST

In [310]:
genome_129_CAST_for_output = [SeqRecord(Seq(sequence), id=key, description="") for key, sequence in genome_129_CAST.items()]

with open('/DATA/projects/bvs_alleles/sox2_rcmc/custom_genomes/CM2266_129_CAST_custom.fa', 'w') as handle:
    SeqIO.write(genome_129_CAST_for_output, handle, 'fasta')

### Cell line 6: LP-161kb_Sox2P_CDS_mChDel (CM2291)

In [311]:
genome_129_CAST = {}

for record in SeqIO.parse('/DATA/projects/bvs_alleles/GRCm38_129_CAST_snpsonly.fa', 'fasta'):
    genome_129_CAST[record.id] = str(record.seq)

In [312]:
seq_before_lp161_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:-34598480'))
seq_lp161_129 = landing_pad_161kb_sox2p_cds
seq_between_lp161_lp116_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34598480-34644035'))
seq_lp116_129 = landing_pad_116kb_nans
seq_between_lp116_sox2_deletion_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34644035-34647660'))
seq_sox2_deletion_129 = sox2_deletion
seq_after_sox2_deletion_129 = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_129:34652613-'))

custom_chr3_129 = [seq_before_lp161_129, seq_lp161_129, seq_between_lp161_lp116_129, seq_lp116_129, 
                   seq_between_lp116_sox2_deletion_129, seq_sox2_deletion_129, seq_after_sox2_deletion_129]
custom_chr3_129 = ''.join(custom_chr3_129)

In [313]:
genome_129_CAST['3_129'] = custom_chr3_129

In [314]:
seq_before_lp161_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:-34598480'))
seq_lp161_CAST = landing_pad_161kb_nans
seq_between_lp161_lp116_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34598480-34644035'))
seq_lp116_CAST = landing_pad_116kb_nans
seq_between_lp116_egfp_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34644035-34651372'))
seq_egfp_CAST = sox2_egfp
seq_after_egfp_CAST = extract_sequence_for_region(genome_129_CAST, split_regions('chr3_CAST:34651375-'))

custom_chr3_CAST = [seq_before_lp161_CAST, seq_lp161_CAST, seq_between_lp161_lp116_CAST, seq_lp116_CAST, 
                   seq_between_lp116_egfp_CAST, seq_egfp_CAST, seq_after_egfp_CAST]
custom_chr3_CAST = ''.join(custom_chr3_CAST)

In [315]:
genome_129_CAST['3_CAST'] = custom_chr3_CAST

In [317]:
genome_129_CAST_for_output = [SeqRecord(Seq(sequence), id=key, description="") for key, sequence in genome_129_CAST.items()]

with open('/DATA/projects/bvs_alleles/sox2_rcmc/custom_genomes/CM2291_129_CAST_custom.fa', 'w') as handle:
    SeqIO.write(genome_129_CAST_for_output, handle, 'fasta')

# Shift coordinates of the capture regions

Original coordinates from the RCMC paper:
* Sox2 mm10 capture region: chr3:33750000-35650000

New coordinates for Sox2 capture region:
* Sox2 custom capture region: chr3:33750000-35659386

New coordinates for insertions/deletions, Sox2 gene and SCR:
* landing_pad_161kb: chr3:34598480-34603292 
* landing_pad_161kb (CDS part): chr3_129:34601113-34602127
* landing_pad_116kb: chr3:34648847-34652698
* sox2_gene = chr3:34658658-34661124
* sox2_deletion: chr3:34656323-34661999
* SCR: chr3:34762792-34775778